In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### I want to use my code to solve the problem from V. Choi - *Different Adiabatic Quantum Optimization Algorithms for the NP-Complete Exact Cover and 3SAT Problems*

In [ ]:
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\Set Graph.png")
img2=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\Set Graph caption.png")
plt.figure(figsize=(10,6)); plt.imshow(img1);plt.axis('off')
plt.figure(figsize=(15,9)); plt.imshow(img2);plt.axis('off')
plt.show('all')

In this problem we want to return the set of subsets which provide exact coverage

In [ ]:
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\exact_cover.png")
img2=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\exact_cover2.png")
plt.figure(figsize=(12,6)); plt.imshow(img1);plt.axis('off')
plt.figure(figsize=(12,6)); plt.imshow(img2);plt.axis('off')
plt.show('all')

Here had to slightly adapt that and did my own derivation to arrive at the hamiltonian of the problem:
\begin{equation}
Hp =MI+ \frac{1}{2}\sum_{i} B_{i}\sigma_{i}^{z}+\frac{1}{2}\sum_{(i,j); i<j} I_{ij}\sigma_{i}^{z}\sigma_{j}^{z}
\end{equation}
where M is the number of clauses and $B_{i}$ and $I_{ij}$ are as above.

Lets first see if we can make the hamiltonian whose lowest energy eigenstate is the solution to the problem

In [ ]:
# this is a function I can use to see the bit solution to the problem
sigma_z = np.matrix('1. 0.; 0. -1.')

def sigma_zi(i,N):
    '''This returns the equivalent sigma_z for the ith qubit in the new Hilbert space given that there are N qubits'''
    return np.kron(np.eye(2**i), np.kron(sigma_z,np.eye(2**(N-(i+1)))))

def bit_values(qubits,v_final):
    '''this function takes the vector in the hamiltonian space and turns it into an expectation bit value'''
    bits = np.zeros(qubits)
    for i in range(qubits):
        H = sigma_zi(i,qubits)
        bits[i] = 0.5*(1+v_final.getH()*H*v_final)
    return bits



In [ ]:

# we start by listing the clauses from the EC3 problem in the V.Choi paper
clauses = [[1,2,3],[1,2,4],[3,4,5],[1,3,6],[2,6,7]]

def f_cost(clauses,v):
    '''this functuon evaluates the 'cost' of the bit assignment v. it is the cost function of the paper above'''
    f = 0
    for i,c in enumerate(clauses):
        f+= (v[c[0]-1]+v[c[1]-1]+v[c[2]-1]-1)**2
    return f

# here I want to run the cost function on all possible bit assignments to check that the clauses lead to the correct solution
costs = []
assignment = []
for i1 in [0,1]:
    for i2 in [0,1]:
        for i3 in [0,1]:
            for i4 in [0,1]:
                for i5 in [0,1]:
                    for i6 in [0,1]:
                        for i7 in [0,1]:
                            costs.append(f_cost(clauses,np.asarray([i1,i2,i3,i4,i5,i6,i7])))
                            assignment.append(np.asarray([i1,i2,i3,i4,i5,i6,i7]))
costs = np.asarray(costs)

print assignment[costs.argmin()]


        

This returned the correct result

Below I try to construct a hamiltonian function that takes the clauses and returns the hamiltonian which is equivalent to them. 
I also costruct a function that takes the bit assignment and produces the vector for this in the hamiltonian space. 
Finally I check if the hamiltonian function gives back the same 'cost' as the cost function above

In [ ]:
bitsoln = np.asarray([1,0,0,0,1,0,1]) # this is the sultion to the EC3 problem presented in the V.Choi paper

def make_eigen_vector(bits):
    '''this function takes a string of bit assignments and turns them into a vector in the hamiltonian space'''
    neg = np.matrix('0. 1.')
    pos = np.matrix('1. 0.')
    vecs = [neg,pos]
    vec = vecs[bits[0]]
    for i in range(1,len(bits)):
        vec = np.kron(vec,vecs[bits[i]])
    return vec


def H_clauses (qubits,clauses): 
        '''constructs the hamiltonian from clauses. Follows the outline in the paper by Altshuler'''
        #initiate the hamiltonians giving them the right dimensions
        Hp = np.matrix(np.eye(2**qubits)*0.)
        Hp+= np.matrix(np.eye(2**qubits)*len(clauses))# this is the MI part
        
        # find the coefficients needed.
        #h_i is the number of times the bit x_i apears in the clauses
        h_i = [0.]*qubits
        for i in range(qubits):
            for c in clauses:
                if (i+1) in c:
                    h_i[i]+=1.
        # J_ij is the number of times the bits x_i and x_j appear in the same clause
        J_ij =[]
        for i in range(qubits-1):
            J=[]
            for j in range(i+1,qubits,1):
                n = 0.
                for c in clauses: 
                    if (i+1) in c and (j+1) in c:
                        n+= 1
                J.append(n)
            J_ij.append(J)
            
        # loop through adding pieces to construct the hamiltonian
        for i in range(qubits):
            Hp += 0.5*h_i[i]*sigma_zi(i,qubits)
            # the following part adds the interaction between all qubits. There is 1 interaction coefficient for each pairing
            if i == qubits-1: break
            for j in range(len(J_ij[i])): 
                Hp += 0.5*J_ij[i][j]*sigma_zi(i,qubits)*sigma_zi(i+(j+1),qubits)  
        return Hp

Hp = H_clauses (7,clauses)

# initate data stores so that we can compare the cost function and cost hamiltonian results
costs_f = []
costs_H = []
assignment = []
for i1 in [0,1]:
    for i2 in [0,1]:
        for i3 in [0,1]:
            for i4 in [0,1]:
                for i5 in [0,1]:
                    for i6 in [0,1]:
                        for i7 in [0,1]:
                            bits = np.asarray([i1,i2,i3,i4,i5,i6,i7]) # set the bit assignment
                            v = make_eigen_vector(bits) # construct the eigenvector equivalent
                            costs_f.append(f_cost(clauses,bits)) # store functional cost
                            costs_H.append(np.trace(v*Hp*np.transpose(v))) # store hamiltonian cost
                            assignment.append(bits) # store bit assignment
costs_f = np.asarray(costs_f)
costs_H = np.asarray(costs_H)
print costs_f
print costs_H
print costs_f-costs_H

In [ ]:
#check what the eigenvalues are of this hamiltonian, compare these to the cost functions of different assignments
print np.linalg.eigh(Hp)[0]
print np.sort(costs_f)
if np.array_equal(np.linalg.eigh(Hp)[0]-np.sort(costs_f),np.zeros(len(costs_f))): print 'these arrays are equal'

In [ ]:
# this bit is just to check if the eigenvalues from the above hamiltonian are simply ones in all positions of the basis
# ie they form a simple linear basis
x = np.linalg.eigh(Hp)

pos = []
for i in range(len(x[1])): 
    pos.append(x[1][i].argmax())
pos.sort()
pos = np.asarray(pos)
if np.array_equal(pos,np.arange(len(pos))): print 'True, the eigenvalues from the above hamiltonian are simply ones in all positions of the basis'
print pos

In [ ]:
def  EC3_hamiltonian_params(qubits,clauses):
        '''returns the h/2 and J/2 parameters needed to construst the hamiltonian in the EC3 problem'''
        # find the coefficients needed.
        #h_i is the number of times the bit x_i apears in the clauses divided by 2
        h_i = [0.]*qubits
        for i in range(qubits):
            for c in clauses:
                if (i+1) in c:
                    h_i[i]+=0.5
        # J_ij is the number of times the bits x_i and x_j appear in the same clause didvided by 2
        J_ij =[]
        for i in range(qubits-1):
            J=[]
            for j in range(i+1,qubits,1):
                n = 0.
                for c in clauses: 
                    if (i+1) in c and (j+1) in c:
                        n+= 0.5
                J.append(n)
            J_ij.append(J)
        return h_i,J_ij

In [ ]:
%matplotlib inline
clauses = [[1,2,3],[1,2,4],[3,4,5],[1,3,6],[2,6,7]]
qubits = 7
h,J = EC3_hamiltonian_params(qubits,clauses)
a_choi = mylib.Anneal(qubits,[h,J],T=100000,points = 10000)
a_choi.Hp = H_clauses(qubits,clauses) # this part was needed because my Anneal class doesn't allow MI to be added to Hp
a_choi.run()
a_choi.show_results()

In [ ]:
bit_values(qubits,a_choi.problem_x0)

Converged as expected

### To make sure that this is working entirely lets set up another EC3 problem

In [ ]:
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\my_ec3.png")
plt.figure(figsize=(10,6)); plt.imshow(img1);plt.axis('off')
plt.show('all')

In structure this is identical to the problem above except that the clauses here are:
\begin{equation}
c_{1} = x_{2}\lor x_{6} \lor x_{7}
\\
c_{2} = x_{2}\lor x_{4} \lor x_{5}
\\
c_{3} = x_{1}\lor x_{5} \lor x_{8}
\\
c_{4} = x_{4}\lor x_{7} \lor x_{8}
\\
c_{5} = x_{1}\lor x_{3} \lor x_{7}
\end{equation}
and there are multiple solutions

In [ ]:
my_clauses = [[2,6,7],[2,4,5],[1,5,8],[4,7,8],[1,3,7]]

Hp = H_clauses (8,my_clauses)

# initate data stores so that we can compare the cost function and cost hamiltonian results
costs_f = []
costs_H = []
assignment = []
for i1 in [0,1]:
    for i2 in [0,1]:
        for i3 in [0,1]:
            for i4 in [0,1]:
                for i5 in [0,1]:
                    for i6 in [0,1]:
                        for i7 in [0,1]:
                            for i8 in [0,1]:
                                bits = np.asarray([i1,i2,i3,i4,i5,i6,i7,i8]) # set the bit assignment
                                v = make_eigen_vector(bits) # construct the eigenvector equivalent
                                costs_f.append(f_cost(my_clauses,bits)) # store functional cost
                                costs_H.append(np.trace(v*Hp*np.transpose(v))) # store hamiltonian cost
                                assignment.append(bits) # store bit assignment
costs_f = np.asarray(costs_f)
costs_H = np.asarray(costs_H)
print costs_f
print costs_H
print costs_f-costs_H
print 'solutions are as follows:\n'
for i in list(np.where(costs_f==0)[0]):
    print assignment[i]

In [ ]:
%matplotlib inline

qubits = 8
h,J = EC3_hamiltonian_params(qubits,my_clauses)
a_myec3 = mylib.Anneal(qubits,[h,J],T=100000,points = 1000)
a_myec3.Hp = H_clauses(qubits,my_clauses) # this part was needed because my Anneal class doesn't allow MI to be added to Hp
a_myec3.run()
a_myec3.show_results()

In [ ]:
% matplotlib q

The code is messing up very late in the run time but below I have done a check to see what the bit soltuion would be to the problem_x0 and it is as expected. It gives a superposition of the 3 valid answers

In [ ]:
print 'The possible solutions to measure based on the problem x0 are: \n'
for i in np.where(a_myec3.problem_x0 >0.)[0].tolist()[0]:
    str = i*'0; '+ '1.;'+(256-i-1)*'0;'
    print bit_values(8,np.matrix(str[:-1]))

The eigenalues are not converging for s=0.979897989799

In [ ]:
s_nc = 0.979897989799
AB = mylib.linear_schedule(10000)
ss = np.linspace(0,1,10000)

AB =AB[:,ss>0.97]
ss = ss[ss>0.97]
for i,s in enumerate(ss):
    print i,s
    H_nc = -AB[0][i]*a_myec3.Hb+AB[1][i]*a_myec3.Hp
    eigvals, eigvecs =np.linalg.eigh(H_nc)

### You need to do some work to the anneal class. It's not working perfectly because of the hamiltonian being too specific. You should probably have this as an input and also feed out the bit values rather than the eigenvector